## Fake News Classifier Using LSTM
#### Dataset:https://www.kaggle.com/c/fake-news/data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df= df.dropna()

In [5]:
 X = df.drop('label',axis=1)

In [6]:
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
## Vocabulary size/ Dictionary
voc_size = 5000

## Onehot Representation

In [12]:
messages=X.copy()

In [13]:
messages.reset_index(inplace=True)

In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [17]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[480, 4257, 3405, 3193, 3666, 3156, 4930, 2715, 4288, 3324],
 [788, 4504, 4086, 95, 4086, 3514, 4124],
 [4583, 1429, 1949, 4932],
 [3585, 58, 3024, 1078, 1763, 4021],
 [1808, 4086, 4163, 4530, 3935, 713, 4086, 3442, 3233, 3882],
 [4766,
  370,
  613,
  3051,
  3441,
  2349,
  4749,
  2304,
  4027,
  3118,
  1889,
  2219,
  640,
  3467,
  4124],
 [2608, 922, 3008, 3475, 4875, 3574, 4097, 732, 4918, 2523, 1821],
 [4435, 1905, 4522, 1867, 1786, 2520, 2349, 4822, 4918, 2523, 1821],
 [58, 1671, 1776, 4106, 4721, 4711, 2605, 4523, 2349, 3697],
 [2498, 2885, 4919, 4905, 654, 1031, 2297, 823],
 [3154, 4657, 4895, 511, 2028, 3381, 1290, 1522, 3402, 2505, 1883],
 [1078, 3928, 3666, 4711, 2349, 1786],
 [971, 741, 1792, 281, 1123, 1016, 601, 202, 1307],
 [192, 1649, 3082, 3432, 3389, 3451, 585, 4918, 2523, 1821],
 [4050, 3926, 3380, 1664, 3297, 4918, 2523, 1821],
 [4995, 1524, 647, 4508, 2965, 4702, 2926, 414, 2517, 1286],
 [3164, 569, 4504],
 [3059, 4549, 437, 1017, 2349, 4923, 654, 4124],
 [423

## Embedding Representation

In [18]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2715 4288 3324]
 [   0    0    0 ... 4086 3514 4124]
 [   0    0    0 ... 1429 1949 4932]
 ...
 [   0    0    0 ... 4918 2523 1821]
 [   0    0    0 ... 3599 4140 1685]
 [   0    0    0 ... 2014 3203 3830]]


In [19]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  480,
       4257, 3405, 3193, 3666, 3156, 4930, 2715, 4288, 3324])

In [20]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [28]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

### Model Training

In [33]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 48s 52ms/step - loss: 0.3331 - accuracy: 0.8424 - val_loss: 0.1981 - val_accuracy: 0.9148
Epoch 2/10
192/192 [==============================] - 5s 27ms/step - loss: 0.1404 - accuracy: 0.9450 - val_loss: 0.1959 - val_accuracy: 0.9186
Epoch 3/10
192/192 [==============================] - 5s 27ms/step - loss: 0.1024 - accuracy: 0.9620 - val_loss: 0.2150 - val_accuracy: 0.9115
Epoch 4/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0763 - accuracy: 0.9721 - val_loss: 0.2426 - val_accuracy: 0.9056
Epoch 5/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0589 - accuracy: 0.9793 - val_loss: 0.2687 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0437 - accuracy: 0.9860 - val_loss: 0.3340 - val_accuracy: 0.9056
Epoch 7/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0312 - accuracy: 0.9898 - val_loss: 0.4015 - val_accuracy: 0.906

## Performance Metrics And Accuracy

In [40]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

In [41]:
from sklearn.metrics import confusion_matrix

In [42]:
confusion_matrix(y_test,y_pred)

array([[3125,  294],
       [ 277, 2339]], dtype=int64)

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9053852526926264

In [ ]:
## Adding Dropout
from tensorflow.kera.layers import Dropout
# Creating model
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())